# Check problematic raw data files

In [1]:
%load_ext autoreload
%autoreload 2
%cd D:\nine-euro-ticket-de

D:\nine-euro-ticket-de


In [5]:
# Load libs
import pandas as pd
import os
import shutil
from tqdm import tqdm

In [4]:
file_path = 'E:/raw_data_de/raw_data_de_2019/08/18/locations-13-part0001.csv.gz'
selectedcols = ['timestamp', 'device_aid', 'latitude', 'longitude', 'location_method']
df = pd.read_csv(file_path, sep='\t', compression='gzip', usecols=selectedcols)

C:\Users\yuanlia\AppData\Local\Temp\ipykernel_17604\2235260106.py:3: DtypeWarning: Columns (0,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', compression='gzip', usecols=selectedcols)


In [8]:
for column in (df.columns[0], df.columns[3], df.columns[4]):
    # Determine the most common type in the column
    most_common_type = df[column].apply(type).mode()[0]
    
    # Filter out rows that don't match the most common type
    print(column, len(df))
    df = df[df[column].apply(lambda x: isinstance(x, most_common_type))]
    print(column, 'after', len(df))

timestamp 5555870
timestamp after 5523102
longitude 5523102
longitude after 5523102
location_method 5523102
location_method after 5523102


In [9]:
df.to_csv(file_path, sep='\t', compression='gzip', index=False)

## Merge group folders

In [1]:
import os
import shutil

base_dir = 'D:/MAD_dbs/raw_data_de/format_parquet'  # Change this to the path of your base folder

for x in range(301):  # 0 to 300 inclusive
    source_folder = os.path.join(base_dir, f'grp_{x}.0')
    target_folder = os.path.join(base_dir, f'grp_{x}')
    
    if os.path.exists(source_folder) and os.path.exists(target_folder):
        # Move each file in the source folder to the target folder
        for filename in os.listdir(source_folder):
            source_file = os.path.join(source_folder, filename)
            target_file = os.path.join(target_folder, filename)
            
            # If a file with the same name exists, decide how to handle. Here we overwrite it.
            if os.path.exists(target_file):
                print(f"Overwriting: {target_file}")
                os.remove(target_file)
            
            shutil.move(source_file, target_folder)
        
        # Delete the source folder after moving all files
        shutil.rmtree(source_folder)
        print(f"Deleted folder: {source_folder}")
    else:
        print(f"One or both of the folders do not exist: {source_folder}, {target_folder}")

Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_0.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_1.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_2.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_3.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_4.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_5.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_6.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_7.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_8.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_9.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_10.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_11.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_12.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_13.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_14.0
Deleted folder: D:/MAD_dbs/raw_data_de/format_parquet\grp_15.0
De

In [3]:
year, m, d = 2019, '09', '19'
files2check = [f'D:/MAD_dbs/raw_data_de/format_parquet/grp_{grp}/{year}_{m}_{d}_1.parquet' for grp in range(139, 150)] +\
    [f'D:/MAD_dbs/raw_data_de/format_parquet/grp_{grp}/{year}_{m}_{d}_2.parquet' for grp in range(139, 150)]
prob_files = []
for filename in tqdm(files2check, desc='Going through files'):
    try:
        df = pd.read_parquet(filename, engine='fastparquet')
    except:
        prob_files.append(filename)

Going through files: 100%|██████████| 22/22 [00:53<00:00,  2.44s/it]


In [4]:
prob_files

['D:/MAD_dbs/raw_data_de/format_parquet/grp_139/2019_09_19_2.parquet']

## Remove low-precision records

In [30]:
data_folder = 'D:\\MAD_dbs\\raw_data_de\\format_parquet'
target_folder = 'D:\\MAD_dbs\\raw_data_de\\format_parquet_r'
paths = [x[0] for x in os.walk(data_folder)]
paths = paths[1:]
paths2raw = {int(x.split('_')[-1]): x for x in paths}

In [35]:
def num_check(x):
    prec = str(x).split('.')[-1]
    if len(prec) <= 2:
        return 1
    else:
        p = int(str(x * 100).split('.')[-1])
        if p == 0:
            return 1
        else:
            return 0


def low_precision_check(x, y):
    if (num_check(x) == 1) & (num_check(y) == 1):
        return 1
    else:
        return 0

def low_precision_remove(file, batch):
    df = pd.read_parquet(os.path.join(paths2raw[batch], file))
    L = len(df)
    df.loc[:, 'low_precision'] = df.apply(lambda row: low_precision_check(row['latitude'], row['longitude']), axis=1)
    df = df.loc[df.low_precision != 1].drop(columns='low_precision')
    low_share = len(df) / L
    dirs = [x[0] for x in os.walk(target_folder)]
    target_dir = os.path.join(target_folder, 'grp_' + str(batch))
    if target_dir not in dirs:
        os.makedirs(target_dir)
    df.to_parquet(target_dir + f'\\{file}')
    return low_share

In [32]:
share_dict = dict()

In [33]:
batch = 0
files = [x[2] for x in os.walk(paths2raw[batch])]
files = files[0] # [os.path.join(paths2raw[batch], x) for x in files[0]]

In [34]:
share_dict[batch] = [low_precision_remove(file, batch) for file in tqdm(files)]


  0%|          | 0/873 [00:00<?, ?it/s]

created folder :  D:\MAD_dbs\raw_data_de\format_parquet_r\grp_0



  0%|          | 0/873 [03:44<?, ?it/s]3.28s/it]


KeyboardInterrupt: 

In [38]:
df_stats = pd.read_parquet('D:\\MAD_dbs\\raw_data_de\\format_parquet_r_stats\\low_prec_share_0.parquet')
df_stats.describe()

,low_prec_share,grp
count,873.000000,873.0
mean,0.844271,0.0
std,0.107872,0.0
min,0.557842,0.0
25%,0.750339,0.0
50%,0.892680,0.0
75%,0.927831,0.0
max,1.000000,0.0


## Merge two batches

In [6]:
def create_directories(destination_base, num_groups):
    for i in range(num_groups):
        dest_dir = os.path.join(destination_base, f'grp_{i}')
        os.makedirs(dest_dir, exist_ok=True)

def copy_files(source_base1, source_base2, destination_base, num_groups):
    for i in range(num_groups):
        group_dir = f'grp_{i}'
        
        # Define source directories
        source_dir1 = os.path.join(source_base1, group_dir)
        source_dir2 = os.path.join(source_base2, group_dir)
        
        # Define destination directory
        dest_dir = os.path.join(destination_base, group_dir)
        
        # Copy files from the first source directory
        if os.path.exists(source_dir1):
            for filename in os.listdir(source_dir1):
                source_file = os.path.join(source_dir1, filename)
                dest_file = os.path.join(dest_dir, filename)
                shutil.copy2(source_file, dest_file)
        
        # Copy files from the second source directory
        if os.path.exists(source_dir2):
            for filename in os.listdir(source_dir2):
                source_file = os.path.join(source_dir2, filename)
                dest_file = os.path.join(dest_dir, filename)
                shutil.copy2(source_file, dest_file)

# Define paths
destination_base = 'D:/MAD_dbs/raw_data_de/format_parquet2stops'
source_base1 = 'D:/MAD_dbs/raw_data_de/format_parquet_r'
source_base2 = 'D:/MAD_dbs/raw_data_de/format_parquet_br'
num_groups = 300

# Create directories and copy files
create_directories(destination_base, num_groups)
copy_files(source_base1, source_base2, destination_base, num_groups)

KeyboardInterrupt: 

In [7]:
# File location and structure
data_folder = 'D:\\MAD_dbs\\raw_data_de\\format_parquet_r'
paths = [x[0] for x in os.walk(data_folder)]
paths = paths[1:]
file_paths_dict = dict()
for path in paths:
    bt = int(path.split('_')[-1])
    files = os.listdir(path)
    file_paths = [os.path.join(path, f) for f in files]
    file_paths_dict[bt] = file_paths   # 300 groups of users

In [27]:
file_paths_dict = dict()
data_folder1 = 'D:\\MAD_dbs\\raw_data_de\\format_parquet_r'
paths1 = [data_folder1 + f'\\grp_{x}' for x in range(0, 300)]
data_folder2 = 'D:\\MAD_dbs\\raw_data_de\\format_parquet_br'
paths2 = [data_folder2 + f'\\grp_{x}' for x in range(0, 300)]
for path1, path2, grp in zip(paths1, paths2, range(0, 300)):
    files1 = os.listdir(path1)
    file_paths1 = [os.path.join(path1, f) for f in files1]
    files2 = os.listdir(path2)
    file_paths2 = [os.path.join(path2, f) for f in files2]
    file_paths_dict[grp] = file_paths1 + file_paths2   # 300 groups of users

In [28]:
len(set(file_paths_dict[299]))

873